# Prototype 02 > Experiment 01

In [1]:
executed_yet = False

In [2]:
import os

if not executed_yet:
    executed_yet = True
    original_working_directory_path = os.getcwd()
    os.chdir(os.path.join(original_working_directory_path, "../.."))
    root_working_directory_path =  os.getcwd()
    
print(f'Original working directory: {original_working_directory_path}')
print(f'Current working directory: {root_working_directory_path}')

Original working directory: /Users/jankreischer/Library/Mobile Documents/com~apple~CloudDocs/Master-Thesis/Code/prototypes/prototype_03
Current working directory: /Users/jankreischer/Library/Mobile Documents/com~apple~CloudDocs/Master-Thesis/Code


## --- Dependencies ---

In [3]:
# Standard Dependencies
import sys
import os
import numpy as np
from time import time

In [5]:
# Global Dependencies
from src.functions import calculate_balance_metrics
from src.custom_types import Behavior, MTDTechnique,  Execution, Evaluation, actions, mitigated_by, normal_afterstates
from src.data_provider import DataProvider
from src.evaluation_utils import plot_learning, seed_random, get_pretrained_agent, evaluate_agent, evaluate_agent_on_afterstates
from src.autoencoder import AutoEncoder, RMSELoss

/Users/jankreischer/opt/anaconda3/envs/FedRL-for-IT-Sec/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
#from prototypes.prototype_02.agent import Agent
#from prototypes.prototype_02.client import Client
#from prototypes.prototype_02.server import Server
#from prototypes.prototype_02.experiment import Experiment

In [6]:
import pandas as pd
decision_states_dataset = pd.read_csv('prototypes/prototype_03/dataset-02_decision-state-samples.csv')
print(len(decision_states_dataset))
after_states_dataset = pd.read_csv('prototypes/prototype_03/dataset-02_after-state-samples.csv')
print(len(after_states_dataset))
dataset = pd.concat([decision_states_dataset, after_states_dataset], axis=0)

17332
60549


In [7]:
decision_states_dataset["mtd"].unique()

array(['None'], dtype=object)

In [8]:
# 3 Status Features
time_status_columns = ['time', 'timestamp', 'seconds']
try:
    dataset.drop(time_status_columns, inplace=True, axis=1)
except:
    print("All time status features are removed from the dataset")
assert len(dataset.columns) == 99

In [9]:
from fast_ml.feature_selection import get_constant_features

constant_features = set(get_constant_features(dataset, threshold=0.99, dropna=False)['Var'])
print(constant_features)
try:
    dataset.drop(constant_features, inplace=True, axis=1)
except:
    print("All constant features are removed from the dataset")
print(dataset.shape)

{'cpuNice', 'tasksStopped', 'udp:udp_fail_queue_rcv_skb', 'dma_fence:dma_fence_init', 'cpuHardIrq', 'clk:clk_set_rate', 'alarmtimer:alarmtimer_start', 'cachefiles:cachefiles_create', 'cachefiles:cachefiles_lookup', 'alarmtimer:alarmtimer_fired', 'cachefiles:cachefiles_mark_active', 'connectivity'}
(77881, 87)


In [103]:
normal_afterstate_strings = [
    ("Behavior.ROOTKIT_BDVL", "MTDTechnique.ROOTKIT_SANITIZER"),
    ("Behavior.ROOTKIT_BEURK", "MTDTechnique.ROOTKIT_SANITIZER"),
    ("Behavior.RANSOMWARE_POC", "MTDTechnique.RANSOMWARE_DIRTRAP"),
    ("Behavior.RANSOMWARE_POC", "MTDTechnique.RANSOMWARE_FILE_EXT_HIDE"),
    ("Behavior.CNC_BACKDOOR_JAKORITAR", "MTDTechnique.CNC_IP_SHUFFLE"),
    ("Behavior.CNC_THETICK", "MTDTechnique.CNC_IP_SHUFFLE"),
    ("Behavior.CNC_OPT1", "MTDTechnique.CNC_IP_SHUFFLE"),
    ("Behavior.CNC_OPT2", "MTDTechnique.CNC_IP_SHUFFLE"),
]

In [104]:
dataset["behavior"].unique()

array(['Behavior.NORMAL', 'Behavior.RANSOMWARE_POC',
       'Behavior.ROOTKIT_BDVL', 'Behavior.CNC_BACKDOOR_JAKORITAR',
       'Behavior.ROOTKIT_BEURK', 'Behavior.CNC_THETICK',
       'Behavior.CNC_OPT1', 'Behavior.CNC_OPT2'], dtype=object)

In [105]:
normal_label = 0
abnormal_label = 1

def is_normal(sample):  
    behavior = sample.behavior 
    mtd = sample.mtd
    if behavior == "Behavior.NORMAL":
        label = normal_label
    elif (str(behavior), str(mtd)) in normal_afterstate_strings:
        label = normal_label
    else:
        label = abnormal_label
    return label

In [106]:
# Add a label if a state should be considered normal or not
dataset['is_normal'] = dataset.apply(lambda sample: is_normal(sample), axis=1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# Scaling
fit_normal_behavior_only = True
standard_scaling = False
if standard_scaling:
    scaler = StandardScaler()
else:
    scaler = MinMaxScaler()
    
print(f"Using {scaler}")

if fit_normal_behavior_only: 
    scaler.fit(dataset[dataset['behavior'] == "Behavior.NORMAL"].values[:,:-2])
else: 
    scaler.fit(dataset.values[:,:-1])

scaled_dataset = pd.DataFrame(scaler.transform(dataset.values[:,:-2]), columns=dataset.columns.drop(["behavior", "mtd"]), index=dataset.index)
scaled_dataset["behavior"] = dataset["behavior"]
scaled_dataset["mtd"] = dataset["mtd"]

In [108]:
from sklearn.preprocessing import MinMaxScaler
# Scaling
fit_normal_behavior_only = True
standard_scaling = False
if standard_scaling:
    scaler = StandardScaler()
else:
    scaler = MinMaxScaler()
    
print(f"Using {scaler}")

if fit_normal_behavior_only:
    df = dataset[dataset['is_normal'] == normal_label]

else: 
    df = dataset
    
scaler.fit(df.values[:,:-3])

scaled_dataset = pd.DataFrame(scaler.transform(dataset.values[:,:-3]), columns=dataset.columns.drop(["behavior", "mtd", "is_normal"]), index=dataset.index)
scaled_dataset["behavior"] = dataset["behavior"]
scaled_dataset["mtd"] = dataset["mtd"]
scaled_dataset["is_normal"] = dataset["is_normal"]

Using MinMaxScaler()


In [109]:
from sklearn.model_selection import train_test_split

rl_dataset, ae_dataset = train_test_split(scaled_dataset, train_size=0.5, shuffle=True)
print(len(rl_dataset))
print(len(ae_dataset))

38940
38941


In [110]:
n_total = 0
for behavior in Behavior:
    for mtd in ["None"] + list(MTDTechnique):
        behavior_samples = rl_dataset.loc[(rl_dataset['behavior'] == str(behavior)) & (rl_dataset['mtd'] == str(mtd))]
        n_total+=len(behavior_samples)
        print(f"{behavior}, {mtd} : labeled {behavior_samples['is_normal'].unique()} ({len(behavior_samples)} samples)")
print(f"Contains a total of {n_total} samples.")

Behavior.NORMAL, None : labeled [0] (2036 samples)
Behavior.NORMAL, MTDTechnique.CNC_IP_SHUFFLE : labeled [0] (999 samples)
Behavior.NORMAL, MTDTechnique.ROOTKIT_SANITIZER : labeled [0] (952 samples)
Behavior.NORMAL, MTDTechnique.RANSOMWARE_DIRTRAP : labeled [0] (1073 samples)
Behavior.NORMAL, MTDTechnique.RANSOMWARE_FILE_EXT_HIDE : labeled [0] (996 samples)
Behavior.ROOTKIT_BDVL, None : labeled [1] (845 samples)
Behavior.ROOTKIT_BDVL, MTDTechnique.CNC_IP_SHUFFLE : labeled [1] (325 samples)
Behavior.ROOTKIT_BDVL, MTDTechnique.ROOTKIT_SANITIZER : labeled [0] (1004 samples)
Behavior.ROOTKIT_BDVL, MTDTechnique.RANSOMWARE_DIRTRAP : labeled [1] (707 samples)
Behavior.ROOTKIT_BDVL, MTDTechnique.RANSOMWARE_FILE_EXT_HIDE : labeled [1] (329 samples)
Behavior.ROOTKIT_BEURK, None : labeled [1] (989 samples)
Behavior.ROOTKIT_BEURK, MTDTechnique.CNC_IP_SHUFFLE : labeled [1] (994 samples)
Behavior.ROOTKIT_BEURK, MTDTechnique.ROOTKIT_SANITIZER : labeled [0] (1098 samples)
Behavior.ROOTKIT_BEURK, MTDT

In [20]:
ae_normal_x = ae_dataset.loc[ae_dataset["is_normal"] == 0].drop(["behavior", "mtd", "is_normal"],  axis=1, inplace=False).to_numpy().astype(np.float32) 
print(ae_normal_x.shape)

(14452, 85)


In [111]:
threshold = int(0.6666*len(ae_normal_x))
ae_train_x = ae_normal_x[:threshold]
ae_valid_x = ae_normal_x[threshold:]
print(ae_train_x.shape)
print(ae_valid_x.shape)

(9633, 85)
(4819, 85)


In [122]:
from torch import nn
import torch
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report, accuracy_score
from tabulate import tabulate


class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = torch.nn.MSELoss()
        self.eps = eps
        
    def forward(self, yhat, y):
        loss = torch.sqrt(self.mse(yhat,y) + self.eps)
        return loss
    

class AutoEncoder(torch.nn.Module):
    

    def __init__(self, X_valid, evaluation_data, n_stds=[1], n_hidden_1=64, n_hidden_2=32, activation_function=nn.GELU(), batch_size: int = 64, verbose=False):

        super().__init__()
        
        validation_dataset = torch.utils.data.TensorDataset(
            torch.from_numpy(X_valid).type(torch.float),

        )
        self.validation_data_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=1, shuffle=True, drop_last=True)

        self.y_test = evaluation_data[["is_normal"]].to_numpy().astype(np.float32) 
        #print(f"type(self.y_test): {type(self.y_test)}; self.y_test.shape: {self.y_test.shape}")
        self.X_test = evaluation_data.drop(["behavior", "mtd", "is_normal"],  axis=1, inplace=False).to_numpy().astype(np.float32) 
        #print(f"type(self.X_test): {type(self.X_test)}; self.X_test.shape: {self.X_test.shape}")
        
        self.evaluation_data = evaluation_data
        self.n_stds = n_stds
        
        n_features = X_valid.shape[1]
        
        self.model = nn.Sequential(
            nn.Linear(n_features, n_hidden_1),
            nn.BatchNorm1d(n_hidden_1),
            activation_function,
            nn.Linear(n_hidden_1, n_hidden_2),
            activation_function,
            #nn.Linear(32, 16),
            #activation_function,
            #nn.Linear(16, 32),
            #activation_function,
            nn.Linear(n_hidden_2, n_hidden_1),
            nn.BatchNorm1d(n_hidden_1),
            activation_function,
            nn.Linear(n_hidden_1, n_features),
            activation_function
        )
        
        print(f"Using {n_hidden_2} neurons")
        self.threshold = None
        self.loss_mean = None
        self.loss_standard_deviation = None
        
        self.verbose = verbose

        
    def forward(self, X):
        return self.model(X)
    
    
    def pretrain(self, X_train, optimizer=torch.optim.SGD, loss_function=torch.nn.MSELoss(reduction='mean'), num_epochs: int = 15, batch_size=64, verbose=False):
        
        training_dataset = torch.utils.data.TensorDataset(
            torch.from_numpy(X_train).type(torch.float),
        )
        training_data_loader = torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

        epoch_losses = []
        #for e in tqdm(range(num_epochs), unit="epoch", leave=False):
        for e in range(num_epochs):
            self.train()
            current_losses = []
            for batch_index, (inputs,) in enumerate(training_data_loader):
                optimizer.zero_grad()
                outputs = self.forward(inputs)
                loss = loss_function(inputs, outputs)
                loss.backward()
                optimizer.step()
                current_losses.append(loss.item())
            
            epoch_losses.append(np.average(current_losses))
            if verbose:
                print(f'Training Loss in epoch {e + 1}: {epoch_losses[e]}')
            
        self.analyze_loss()

    '''
    This function uses normal data samles 
    after training the autoencoder to determine
    values that can be considered normal
    for the reconstruction loss based on normal samples
    '''
    def analyze_loss(self):
        losses = []
        
        self.eval() 
        with torch.no_grad():
            loss_function = torch.nn.MSELoss(reduction='sum')
            for batch_index, (inputs,) in enumerate(self.validation_data_loader):
                outputs = self.forward(inputs)
                loss = loss_function(inputs, outputs)
                losses.append(loss.item())
        
        losses = np.array(losses)

        self.loss_mean = losses.mean()
        #print(f"self.loss_mean: {self.loss_mean}")
        #print(f"type(self.loss_mean): {type(self.loss_mean)}; self.loss_mean.shape: {self.loss_mean.shape}")
        self.loss_standard_deviation = losses.std()
        #print(f"self.loss_standard_deviation: {self.loss_standard_deviation}")
        #print(f"type(self.loss_standard_deviation): {type(self.loss_standard_deviation)}; self.loss_standard_deviation: {self.loss_standard_deviation.shape}")

        
    def predict(self, x, n_std):
        test_data = torch.utils.data.TensorDataset(
            torch.from_numpy(x).type(torch.float32)
        )
        test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)

        all_predictions = torch.tensor([])  # .cuda()

        self.eval()
        with torch.no_grad():
            ae_loss = torch.nn.MSELoss(reduction="sum")
            for idx, (batch_x,) in enumerate(test_data_loader):
                model_predictions = self.forward(batch_x)
                model_predictions = ae_loss(model_predictions, batch_x).unsqueeze(0)  # unsqueeze as batch_size set to 1
                all_predictions = torch.cat((all_predictions, model_predictions))

        threshold = self.loss_mean + n_std * self.loss_standard_deviation
        all_predictions = (all_predictions > threshold).type(torch.long)
        return all_predictions.flatten()
    
    
    def predict_deviation(self, x):
        test_data = torch.utils.data.TensorDataset(
            torch.from_numpy(x).type(torch.float32)
        )
        test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)

        prediction_errors = torch.tensor([])
        loss_function = torch.nn.MSELoss(reduction="sum")
        
        self.eval()
        with torch.no_grad():
            
            for batch_index, (inputs,) in enumerate(test_data_loader):
                prediction = self.forward(inputs)
                prediction_error = loss_function(inputs, prediction).unsqueeze(0)  # unsqueeze as batch_size set to 1
                prediction_errors = torch.cat((prediction_errors, prediction_error))

        return prediction_errors
    
    
    def score(self):
        n_std, accuracy = self.accuracy_score(None, None)
        if self.verbose:
            print(f">> Highest validation accuracy achieved {accuracy:.2f} with n_std={n_std} <<")
            self.evaluate(n_std)
        return accuracy
    
    
    def accuracy_score(self, X, y):
        #if not self.threshold:
        #loss_mean, loss_standard_deviation = self.analyze_loss(X)
        #n_stds = np.arange(0.1, 3, 0.1)
        if self.loss_mean == None or self.loss_standard_deviation == None:
              #print("accuracy_score_optimized > accurcy_loss()")
              self.analyze_loss()
    
        best_accuracy = 0
        best_n_std = 0
        #accuracies = []
        y_dev = self.predict_deviation((self.X_test).astype(np.float32))
        for n_std in self.n_stds:
            #print(f"Using n_std: {n_std}")
            y_true = self.y_test
            threshold = self.loss_mean + n_std * self.loss_standard_deviation
            #print(f"threshold: {threshold}")
            #print(f"y_dev: type {type(y_dev)} and shape {y_dev.shape}")
            #print(f"threshold: {type(threshold)} and shape {threshold.shape}")
            y_pred = (y_dev > threshold).type(torch.long).detach().cpu().numpy()
            
            accuracy = accuracy_score(y_true, y_pred)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_n_std = n_std
            #if self.verbose:
            #    print(f"n_std {n_std:.2f} -> accuracy: {accuracy}")

        return best_n_std, best_accuracy
    
    def evaluate(self, n_std=1, tablefmt='pipe'):
        print(f"Using {n_std} n_stds for the decision threshold")
        results = []
        headers=["Behavior", "Type", "After MTD", "Accuracy", "\#Samples"]
        
        y_true_total = np.empty([0])
        y_pred_total = np.empty([0])
        
        for behavior in Behavior:
            for mtd in ["None"] + list(MTDTechnique):
                behavior_samples = self.evaluation_data.loc[(self.evaluation_data['behavior'] == str(behavior)) & (self.evaluation_data['mtd'] == str(mtd))]
                
                y_true= behavior_samples[["is_normal"]].to_numpy().flatten().astype(np.float32) 
                #print(y_true.shape)
                #print(y_true_total.shape)
                y_true_total = np.concatenate((y_true_total, y_true))
                
                X_test = behavior_samples.drop(["behavior", "mtd", "is_normal"],  axis=1, inplace=False).to_numpy().astype(np.float32) 
                
                y_pred = self.predict(X_test, n_std=n_std)
                #print(f"{behavior}, {mtd} : Predicted {len(y_pred)} for {len(X_test)} given samples")
                y_pred_total = np.concatenate((y_pred_total, y_pred))
                
                accuracy = accuracy_score(y_true, y_pred)
            
                n_samples = len(y_true)
                
                if mtd == 'None':
                    state_type = "Decision"
                else:
                    state_type = "After"
                results.append([behavior.name.replace('_', '\_'), state_type, str(mtd), f'{(100 * accuracy):.2f}\%', str(n_samples)])
        
        
        micro_accuracy = accuracy_score(y_true_total, y_pred_total)
        n_samples = len(y_true_total)
        results.append(["MICRO ACCURACY", "", "", f'{(100 * micro_accuracy):.2f}\%', n_samples])
        
        print(tabulate(results, headers=headers, tablefmt=tablefmt))

In [125]:
autoencoder = AutoEncoder(ae_valid_x, rl_dataset, n_hidden_1=64, n_hidden_2=32)

Using 32 neurons


In [126]:
autoencoder.pretrain(ae_train_x, optimizer=torch.optim.Adam(autoencoder.parameters(), lr=1e-4,  weight_decay=0.01), loss_function=RMSELoss(), num_epochs=20, batch_size=64, verbose=True)

Training Loss in epoch 1: 0.4042696422338486
Training Loss in epoch 2: 0.3162392435471217
Training Loss in epoch 3: 0.2343783410390218
Training Loss in epoch 4: 0.18906875337163606
Training Loss in epoch 5: 0.17133437638481458
Training Loss in epoch 6: 0.16079341972867647
Training Loss in epoch 7: 0.15428711950778962
Training Loss in epoch 8: 0.147991261134545
Training Loss in epoch 9: 0.1449591875076294
Training Loss in epoch 10: 0.14097564746936161
Training Loss in epoch 11: 0.13772769649823507
Training Loss in epoch 12: 0.13626951843500137
Training Loss in epoch 13: 0.13502028365929922
Training Loss in epoch 14: 0.132609855979681
Training Loss in epoch 15: 0.13185864662130672
Training Loss in epoch 16: 0.12949390386541684
Training Loss in epoch 17: 0.12845855390032132
Training Loss in epoch 18: 0.1278797531624635
Training Loss in epoch 19: 0.12733520597219466
Training Loss in epoch 20: 0.12665482873717943


In [128]:
autoencoder.evaluate(n_std=2)

Using 2 n_stds for the decision threshold
| Behavior                 | Type     | After MTD                             | Accuracy   |   \#Samples |
|:-------------------------|:---------|:--------------------------------------|:-----------|------------:|
| NORMAL                   | Decision | None                                  | 100.00\%   |        2036 |
| NORMAL                   | After    | MTDTechnique.CNC_IP_SHUFFLE           | 100.00\%   |         999 |
| NORMAL                   | After    | MTDTechnique.ROOTKIT_SANITIZER        | 100.00\%   |         952 |
| NORMAL                   | After    | MTDTechnique.RANSOMWARE_DIRTRAP       | 100.00\%   |        1073 |
| NORMAL                   | After    | MTDTechnique.RANSOMWARE_FILE_EXT_HIDE | 100.00\%   |         996 |
| ROOTKIT\_BDVL            | Decision | None                                  | 0.12\%     |         845 |
| ROOTKIT\_BDVL            | After    | MTDTechnique.CNC_IP_SHUFFLE           | 1.23\%     |         3

In [114]:
import skorch
from skorch import NeuralNet, NeuralNetRegressor
from skorch.scoring import loss_scoring

class StateAnomalyDetector(skorch.NeuralNet):
    def score(self, X, y=None):
        return self.module_.score()
    
    def evaluate(self):
        self.module_.evaluate()

In [115]:
from skorch import NeuralNet, NeuralNetRegressor
from skorch.callbacks import LRScheduler, EarlyStopping

state_anomaly_detector = StateAnomalyDetector(
    module=AutoEncoder,
    
    criterion=RMSELoss(),
    optimizer=torch.optim.SGD,
    optimizer__lr=0.0001,
    optimizer__weight_decay=0.01,
    
    iterator_train__shuffle=True,
    max_epochs=100,
    batch_size=32,
    
    module__X_valid=ae_valid_x,
    module__evaluation_data=rl_dataset,
    #module__n_hidden_1=64,
    #module__n_hidden_2=32,
    #module__activation_function=nn.GELU(),
    #module__n_stds=np.linspace(0.1, , ),
    module__n_stds=np.linspace(0.1, 3, 30),
    module__verbose=True,
    
    verbose=False,
    
    callbacks=[EarlyStopping(patience=5, monitor='valid_loss')]

)

In [118]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'criterion': [RMSELoss, torch.nn.MSELoss, torch.nn.L1Loss()],
    'optimizer': [torch.optim.Adam, torch.optim.SGD, torch.optim.RMSprop],
    #'lr': [0.0001],
    #'max_epochs': [100],
    #'batch_size': [64],
    #'module__activation_function': [torch.nn.Tanh(), torch.nn.GELU(), torch.nn.ReLU(), torch.nn.Sigmoid(), torch.nn.ELU()],
    'module__activation_function': [torch.nn.GELU()],
    'module__n_hidden_1': [64],
    #'module__n_hidden_2': [32, 16, 8],
    'module__n_hidden_2': [32],
     'module__n_stds' : [[0.1, 0.01, 0.001, 0.0001]],
}

ae_grid_search_01 = GridSearchCV(estimator=state_anomaly_detector,
                                 param_grid=param_grid,
                                 n_jobs=1,
                                 cv=2,
                                 verbose=10,
                                 refit=False,
                                 error_score="raise"
                                )
ae_grid_search_01_result = ae_grid_search_01.fit(ae_train_x, ae_train_x)

Fitting 2 folds for each of 9 candidates, totalling 18 fits
[CV 1/2; 1/9] START criterion=<class '__main__.RMSELoss'>, module__activation_function=GELU(approximate='none'), module__n_hidden_1=64, module__n_hidden_2=32, module__n_stds=[0.1, 0.01, 0.001, 0.0001], optimizer=<class 'torch.optim.adam.Adam'>
Using 32 neurons
>> Highest validation accuracy achieved 0.42 with n_std=0.001 <<
| Behavior                 | Type     | After MTD                             | Accuracy   |   \#Samples |
|:-------------------------|:---------|:--------------------------------------|:-----------|------------:|
| NORMAL                   | Decision | None                                  | 100.00\%   |        2036 |
| NORMAL                   | After    | MTDTechnique.CNC_IP_SHUFFLE           | 100.00\%   |         999 |
| NORMAL                   | After    | MTDTechnique.ROOTKIT_SANITIZER        | 100.00\%   |         952 |
| NORMAL                   | After    | MTDTechnique.RANSOMWARE_DIRTRAP       |

KeyboardInterrupt: 

In [64]:
autoencoder.evaluate(n_std=0.5, tablefmt='latex_raw')

Behavior.NORMAL, None : Predicted 2113 for 2113 given samples
Behavior.NORMAL, MTDTechnique.CNC_IP_SHUFFLE : Predicted 986 for 986 given samples
Behavior.NORMAL, MTDTechnique.ROOTKIT_SANITIZER : Predicted 974 for 974 given samples
Behavior.NORMAL, MTDTechnique.RANSOMWARE_DIRTRAP : Predicted 1059 for 1059 given samples
Behavior.NORMAL, MTDTechnique.RANSOMWARE_FILE_EXT_HIDE : Predicted 953 for 953 given samples
Behavior.ROOTKIT_BDVL, None : Predicted 852 for 852 given samples
Behavior.ROOTKIT_BDVL, MTDTechnique.CNC_IP_SHUFFLE : Predicted 334 for 334 given samples
Behavior.ROOTKIT_BDVL, MTDTechnique.ROOTKIT_SANITIZER : Predicted 1030 for 1030 given samples
Behavior.ROOTKIT_BDVL, MTDTechnique.RANSOMWARE_DIRTRAP : Predicted 736 for 736 given samples
Behavior.ROOTKIT_BDVL, MTDTechnique.RANSOMWARE_FILE_EXT_HIDE : Predicted 314 for 314 given samples
Behavior.ROOTKIT_BEURK, None : Predicted 996 for 996 given samples
Behavior.ROOTKIT_BEURK, MTDTechnique.CNC_IP_SHUFFLE : Predicted 976 for 976 giv